# ส่วนที่ 5.1: แนวทางการออกแบบ Power BI Dashboard - ตอนที่ 1

Notebook นี้แสดงแนวทางการออกแบบ Dashboard ใน Power BI สำหรับข้อมูลการจ้างงาน อธิบายหลักการออกแบบ การเลือกใช้กราฟ การอ่านค่าข้อมูล และการเชื่อมต่อกับฐานข้อมูล SQLite

## เนื้อหาครอบคลุม
1. การเชื่อมต่อ Power BI กับฐานข้อมูล SQLite
2. การสร้างความสัมพันธ์ระหว่างตาราง (Data Model)
3. การออกแบบ Dashboard แต่ละหน้า
4. หลักการเลือกใช้ Visualization ที่เหมาะสม
5. การสร้าง Measures และ Calculated Columns ด้วย DAX
6. เทคนิคการสร้าง Dashboard ที่มีประสิทธิภาพ

## 1. การเชื่อมต่อ Power BI กับฐานข้อมูล SQLite

### 1.1 วิธีการเชื่อมต่อ

การเชื่อมต่อ Power BI กับฐานข้อมูล SQLite มีขั้นตอนดังนี้:

1. เปิดโปรแกรม Power BI Desktop
2. คลิกที่ `Get Data` บน Ribbon
3. เลือก `Database` > `SQLite Database`
4. ระบุพาธของไฟล์ฐานข้อมูล (`employee_database.db`)
5. เลือกตารางที่ต้องการนำเข้า หรือเขียน SQL Query เอง
6. มีตารางที่ควรนำเข้า 4 ตาราง คือ
   - `employee_stats` (ข้อมูลหลัก)
   - `region_summary` (ข้อมูลสรุปตามภูมิภาค)
   - `time_summary` (ข้อมูลสรุปตามช่วงเวลา)
   - `province_summary` (ข้อมูลสรุปตามจังหวัด)

### 1.2 ตัวอย่าง SQL Query ที่สามารถใช้ได้

```sql
-- ดึงข้อมูลหลักทั้งหมด
SELECT * FROM employee_stats;

-- ดึงข้อมูลสรุปตามภูมิภาค
SELECT * FROM region_summary;

-- ดึงข้อมูลสรุปตามช่วงเวลา
SELECT * FROM time_summary;

-- ดึงข้อมูลสรุปตามจังหวัด
SELECT * FROM province_summary;
```

### 1.3 ข้อควรระวังในการเชื่อมต่อ

- ต้องติดตั้ง SQLite ODBC Driver ก่อนเชื่อมต่อ
- ควรเชื่อมต่อแบบ Direct Query เมื่อข้อมูลมีขนาดใหญ่
- ควรเชื่อมต่อแบบ Import เมื่อต้องการประสิทธิภาพสูงและข้อมูลไม่เปลี่ยนแปลงบ่อย
- ควรคัดกรองข้อมูลให้นำเข้าเฉพาะข้อมูลที่จำเป็น

## 2. การสร้างความสัมพันธ์ระหว่างตาราง (Data Model)

### 2.1 โครงสร้างความสัมพันธ์

ความสัมพันธ์ระหว่างตารางควรเป็นดังนี้:

1. `employee_stats` เป็นตารางหลัก
   - เชื่อมโยงกับ `region_summary` ผ่านฟิลด์ `YEAR`, `QUARTER`, `REGION`, `SEX`
   - เชื่อมโยงกับ `time_summary` ผ่านฟิลด์ `YEAR`, `QUARTER`
   - เชื่อมโยงกับ `province_summary` ผ่านฟิลด์ `PROVINCE`, `SEX`

### 2.2 การสร้างความสัมพันธ์ใน Power BI

1. เปิดมุมมอง Model View (ไอคอนแสดงความสัมพันธ์ทางด้านซ้าย)
2. ลากฟิลด์จากตารางหนึ่งไปยังอีกตารางเพื่อสร้างความสัมพันธ์
3. กำหนดประเภทความสัมพันธ์ (Cardinality) ให้เหมาะสม
   - Many-to-One: จาก `employee_stats` ไปยังตารางสรุปต่างๆ
4. กำหนดทิศทางการกรอง (Filter Direction) เป็น Both หรือ Single ตามความเหมาะสม

### 2.3 การจัดการข้อมูลวันที่

หากใช้ฟิลด์ `ESTIMATED_DATE` เป็นวันที่โดยประมาณ ควรสร้าง Date Table เพิ่มเติม:

```dax
DateTable = 
CALENDAR(
    DATE(2020, 1, 1),
    DATE(2023, 12, 31)
)
```

และเพิ่มคอลัมน์เพิ่มเติมเช่น ปี ไตรมาส เดือน วัน เพื่อให้สามารถวิเคราะห์ตามช่วงเวลาได้หลากหลายมากขึ้น

## 3. การออกแบบ Dashboard แต่ละหน้า

### 3.1 Dashboard หน้าที่ 1: ภาพรวม (Overview)

**วัตถุประสงค์**: แสดงภาพรวมของข้อมูลการจ้างงานทั้งหมด

**Visualizations ที่ควรใช้**:
1. **Cards** - แสดงตัวเลขสำคัญ:
   - จำนวนพนักงานทั้งหมด
   - จำนวนพนักงานชาย
   - จำนวนพนักงานหญิง
   - อัตราส่วนชาย:หญิง
   - การเปลี่ยนแปลง YoY

2. **Line Chart** - แสดงแนวโน้มการจ้างงานตามช่วงเวลา:
   - แกน X: ปี-ไตรมาส
   - แกน Y: จำนวนพนักงาน
   - สี/Legend: แยกตามเพศ

3. **Donut Chart** - แสดงสัดส่วนพนักงานตามเพศ

4. **Bar Chart** - แสดงจำนวนพนักงานตามภูมิภาค:
   - แกน Y: ภูมิภาค (เรียงจากมากไปน้อย)
   - แกน X: จำนวนพนักงาน

5. **Filled Map** - แสดงจำนวนพนักงานตามจังหวัด:
   - พื้นที่: จังหวัด
   - ความเข้มของสี: จำนวนพนักงาน

6. **Slicers** - ตัวกรองข้อมูล:
   - ปี
   - ไตรมาส
   - ภูมิภาค
   - เพศ

**เหตุผลในการเลือกใช้**:
- **Cards** ใช้เพื่อแสดงตัวเลขสำคัญที่ต้องการเน้นย้ำให้เห็นในทันที
- **Line Chart** เหมาะสำหรับการแสดงแนวโน้มตามช่วงเวลา ทำให้เห็นการเปลี่ยนแปลงอย่างชัดเจน
- **Donut Chart** เหมาะสำหรับการแสดงสัดส่วนเมื่อมีหมวดหมู่ไม่มาก (ในที่นี้คือเพศชายและหญิง)
- **Bar Chart** เหมาะสำหรับการเปรียบเทียบค่าระหว่างหมวดหมู่ (ภูมิภาค) โดยเรียงลำดับจากมากไปน้อย
- **Filled Map** ช่วยให้เห็นการกระจายตัวของข้อมูลตามพื้นที่ได้อย่างชัดเจน

### 3.2 Dashboard หน้าที่ 2: วิเคราะห์ตามภูมิภาค (Regional Analysis)

**วัตถุประสงค์**: วิเคราะห์ข้อมูลการจ้างงานตามภูมิภาคและจังหวัด

**Visualizations ที่ควรใช้**:
1. **Map** - แสดงการกระจายตัวของพนักงานตามพื้นที่:
   - ขนาดของจุด: จำนวนพนักงาน
   - สีของจุด: ภูมิภาค

2. **Clustered Column Chart** - เปรียบเทียบจำนวนพนักงานระหว่างภูมิภาค แยกตามเพศ:
   - แกน X: ภูมิภาค
   - แกน Y: จำนวนพนักงาน
   - กลุ่มแท่ง: แยกตามเพศ

3. **Stacked Bar Chart** - แสดงจังหวัดที่มีพนักงานมากที่สุดในแต่ละภูมิภาค:
   - แกน Y: จังหวัด (Top 3 ของแต่ละภูมิภาค)
   - แกน X: จำนวนพนักงาน
   - สีของแท่ง: ภูมิภาค

4. **Pie Chart** - แสดงสัดส่วนพนักงานในแต่ละภูมิภาค

5. **Matrix** - แสดงข้อมูลเชิงลึกตามจังหวัดและเพศ:
   - แถว: จังหวัด
   - คอลัมน์: เพศ
   - ค่า: จำนวนพนักงาน, % การเปลี่ยนแปลง YoY
   - การจัดกลุ่ม: ตามภูมิภาค

6. **Slicers** - ตัวกรองข้อมูล:
   - ปี
   - ไตรมาส
   - ภูมิภาค (Multi-select / Dropdown)

**เหตุผลในการเลือกใช้**:
- **Map** ช่วยให้เห็นการกระจายตัวเชิงภูมิศาสตร์ได้อย่างชัดเจน
- **Clustered Column Chart** เหมาะสำหรับการเปรียบเทียบระหว่างภูมิภาคและเพศในเวลาเดียวกัน
- **Stacked Bar Chart** เหมาะสำหรับการแสดงจังหวัดที่สำคัญในแต่ละภูมิภาค
- **Pie Chart** ใช้เพื่อแสดงสัดส่วนการกระจายตัวของพนักงานในแต่ละภูมิภาค
- **Matrix** เหมาะสำหรับการแสดงข้อมูลเชิงลึกและเปรียบเทียบในรูปแบบตาราง

### 3.3 Dashboard หน้าที่ 3: แนวโน้มตามเวลา (Time Trend Analysis)

**วัตถุประสงค์**: วิเคราะห์แนวโน้มการจ้างงานตามช่วงเวลา

**Visualizations ที่ควรใช้**:
1. **Area Chart** - แสดงแนวโน้มการจ้างงานตามช่วงเวลา:
   - แกน X: ปี-ไตรมาส
   - แกน Y: จำนวนพนักงาน
   - พื้นที่: แยกตามเพศ

2. **Column Chart** - แสดงอัตราการเปลี่ยนแปลง QoQ:
   - แกน X: ปี-ไตรมาส
   - แกน Y: % การเปลี่ยนแปลง QoQ
   - สี: สีเขียวเมื่อเป็นบวก, สีแดงเมื่อเป็นลบ

3. **Line Chart** - แสดงค่าเฉลี่ยเคลื่อนที่ 4 ไตรมาส:
   - แกน X: ปี-ไตรมาส
   - แกน Y: จำนวนพนักงานเฉลี่ย
   - เส้น: แยกตามภูมิภาค

4. **Scatter Chart** - แสดงความสัมพันธ์ระหว่างจำนวนพนักงานชายและหญิง:
   - แกน X: จำนวนพนักงานชาย
   - แกน Y: จำนวนพนักงานหญิง
   - ขนาดของจุด: จำนวนพนักงานรวม
   - สีของจุด: ปี

5. **Heatmap (Matrix)** - แสดงการเปลี่ยนแปลงตามปี-ไตรมาสและภูมิภาค:
   - แถว: ภูมิภาค
   - คอลัมน์: ปี-ไตรมาส
   - ค่า: % การเปลี่ยนแปลง YoY
   - สี: สีเขียว (เพิ่มขึ้น) ถึงสีแดง (ลดลง)

6. **Line and Clustered Column Chart** - เปรียบเทียบจำนวนพนักงานและอัตราการเปลี่ยนแปลง:
   - แกน X: ปี-ไตรมาส
   - แกน Y1 (Column): จำนวนพนักงาน
   - แกน Y2 (Line): % การเปลี่ยนแปลง YoY

7. **Slicers และ Timeline** - ตัวกรองข้อมูล:
   - Timeline สำหรับปีและไตรมาส
   - ภูมิภาค
   - เพศ

**เหตุผลในการเลือกใช้**:
- **Area Chart** เหมาะสำหรับการแสดงแนวโน้มตามช่วงเวลาพร้อมกับสัดส่วนของแต่ละกลุ่ม
- **Column Chart** เหมาะสำหรับการแสดงอัตราการเปลี่ยนแปลงเป็นรายไตรมาส
- **Line Chart** ค่าเฉลี่ยเคลื่อนที่ช่วยให้เห็นแนวโน้มระยะยาวโดยลดผลกระทบของความผันผวนระยะสั้น
- **Scatter Chart** เหมาะสำหรับการหาความสัมพันธ์ระหว่างตัวแปรสองตัว
- **Heatmap** ช่วยให้เห็นรูปแบบการเปลี่ยนแปลงตามช่วงเวลาและภูมิภาคได้อย่างชัดเจน
- **Line and Clustered Column Chart** ช่วยให้เปรียบเทียบค่าเชิงปริมาณและอัตราการเปลี่ยนแปลงได้ในกราฟเดียวกัน